In [ ]:
import numpy as np
import scipy.stats as sps
import pandas as pd
from statsmodels.stats.proportion import confint_proportions_2indep

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.3)

import warnings
warnings.filterwarnings("ignore")

Компания разработала новую веб-страницу, чтобы попытаться увеличить конверсию, то есть долю пользователей, которые решают оформить подписку и заплатить за продукт компании. Изменение решили проверить с помощью АБ-тестирования, случайно поделив пользователей на группы. Наша цель &mdash; помочь компании понять, следует ли им внедрить эту новую страницу или сохранить старую страницу.

In [7]:
data = pd.read_csv('ab_data.csv')
data

,user_id,timestamp,group,landing_page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1
...,...,...,...,...,...
294475,734608,45:03.4,control,old_page,0
294476,697314,20:29.0,control,old_page,0
294477,715931,40:24.5,treatment,new_page,0
294478,759899,20:29.0,treatment,new_page,0


Посмотрим на соотношение групп.

In [8]:
treatment = data[data['group'] == 'treatment']
control = data[data['group'] == 'control']
print(f'Доля тестовой выборки от всех данных: {np.round(len(treatment) / len(data), 2)}')
print(f'Доля контрольной выборки от всех данных: {np.round(len(control) / len(data), 2)}')

Доля тестовой выборки от всех данных: 0.5
Доля контрольной выборки от всех данных: 0.5


Посчитаем долю пользователей в тесте и в контроле, которые совершили целевое действие, то есть, оформили подписку.

In [9]:
print(f'''Конверсия в тестовой выборке: {np.round(treatment['converted'].mean(), 3)}''')
print(f'''Конверсия в контрольной выборке: {np.round(control['converted'].mean(), 3)}''')

Конверсия в тестовой выборке: 0.119
Конверсия в контрольной выборке: 0.116


Применим t-test к данным. Так как мы знаем, что разделение было случайным, то наблюдения независимы, поэтому применяем t-test для независимых выборок.

In [10]:
alpha = 0.05
stat, pval = sps.ttest_ind(treatment['converted'], control['converted'], equal_var=False)

print('Результаты теста:')
print(f'''p-value критерия: {np.round(pval, 4)}''')
print(f'''Отвергаем ли гипотезу — {pval < alpha}''')

Результаты теста:
p-value критерия: 0.026
Отвергаем ли гипотезу — True


Кажется, что различия в $0.3\%$ незначительные, однако на выборках порядка 150 тыс. человек это довольно много. Посмотрим на разницу в количестве оформивших подписку пользователях в тесте и в контроле. 

In [11]:
print(f'''Кол-во оформивших подписку в тесте: {treatment['converted'].sum()}''')
print(f'''Кол-во оформивших подписку в контроле: {control['converted'].sum()}''')
print(f'''Разница в кол-ве оформивших подписку: {treatment['converted'].sum() - control['converted'].sum()}''')

Кол-во оформивших подписку в тесте: 17514
Кол-во оформивших подписку в контроле: 17116
Разница в кол-ве оформивших подписку: 398


Построим также дов. интервал для прироста в конверсии в подписку.

In [12]:
left_bound, right_bound = confint_proportions_2indep(treatment['converted'].sum(), 
                                                     treatment['converted'].count(),
                                                     control['converted'].sum(), 
                                                     control['converted'].count(),
                                                     method='wald')
print(f'''Дов. интервал для разности конверсий: {np.round(left_bound, 4), np.round(right_bound, 4)}''')

NameError: name 'confint_proportions_2indep' is not defined

Оценим экономический эффект.

Пусть в месяц на сайт заходит 300 тыс. пользователей, подписка стоит 200 рублей. Посчитаем в таком случае прирост в деньгах для компании.

In [13]:
lb_money = 200 * 300_000 * left_bound
rb_money = 200 * 300_000 * right_bound

print(f'''Дов. интервал для прироста в деньгах: {int(np.round(lb_money, -3)), int(np.round(rb_money, -3))}''')

Дов. интервал для прироста в деньгах: (19000, 298000)


Группы А 200 клиентов (50 совершили покупку) 0.25
Группы Б 180 клиентов (30 совершило покупку) 1.666

d = P_A - P_Б


In [5]:
group_a = np.array([1] * 50 + [0] * 150)
group_b = np.array([1] * 30 + [0] * 150)

In [6]:
B = 10000
alpha = 0.05
conf = 1 - alpha

In [7]:
def boot_ci(group1, group2, B, alpha):
    np.random.seed(42)
    
    n1, n2 = len(group1), len(group2)
    boot_dif = []
    
    for _ in range(B):
        
        sample1 = np.random.choice(group1, size=n1, replace=True)
        sample2 = np.random.choice(group2, size=n2, replace=True)
        
        p1 = np.mean(sample1)
        p2 = np.mean(sample2)
        boot_dif.append(p1 - p2)
    
    lower = np.percentile(boot_dif, (alpha / 2) * 100)
    upper = np.percentile(boot_dif, (1 - alpha / 2) * 100)
    
    return lower, upper, boot_dif

In [9]:
ci_lower, ci_upper, diff = boot_ci(group_a, group_b, B, alpha)

In [10]:
print(np.mean(group_a) - np.mean(group_b))

0.08333333333333334


In [11]:
print(ci_lower, ci_upper)

0.0011111111111111183 0.16166666666666665
